# Ventanas deslizantes

In [0]:
#Creación del dataframe
criptos_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
    .option("subscribe", "prom") \
    .load()

criptos_df_string = criptos_df.selectExpr("CAST(value AS STRING)", "timestamp")
criptos_df_string

Out[1]: DataFrame[value: string, timestamp: timestamp]

In [0]:
from pyspark.sql.functions import explode,split,col, json_tuple

#Tomamos los valores presentados en el producer y cambiamos los nombres para su comprensión
criptos_prices = criptos_df_string.select(json_tuple(col("value"), 'prom_btc', 'prom_usd', 'prom_eur',
                                                     'btc', 'usd', 'eur', 'prom_btc_corta', 'prom_usd_corta', 'prom_eur_corta'), col("timestamp")) \
                 .select(col("c0").alias("prom_btc"), col("c1").alias("prom_usd"), col("c2").alias("prom_eur"),
                         col("c3").alias("btc"), col("c4").alias("usd"), col("c5").alias("eur"), 
                         col("c6").alias("prom_btc_corta"), col("c7").alias("prom_usd_corta"),
                         col("c8").alias("prom_eur_corta"), col("timestamp").alias("timestamp"))
print(criptos_prices)

DataFrame[prom_btc: string, prom_usd: string, prom_eur: string, btc: string, usd: string, eur: string, prom_btc_corta: string, prom_usd_corta: string, prom_eur_corta: string, timestamp: timestamp]


In [0]:
#Casteamos los nueve valores anteriores,(que estaban en strings), a floats;
cripto_tab = criptos_prices.withColumn('precio_btc', criptos_prices.prom_btc.cast('float'))
cripto_tab = cripto_tab.withColumn('precio_usd', criptos_prices.prom_usd.cast('float'))
cripto_tab = cripto_tab.withColumn('precio_eur', criptos_prices.prom_eur.cast('float'))


cripto_tab = cripto_tab.withColumn('btc_float', criptos_prices.btc.cast('float'))
cripto_tab = cripto_tab.withColumn('usd_float', criptos_prices.usd.cast('float'))
cripto_tab = cripto_tab.withColumn('eur_float', criptos_prices.eur.cast('float'))

cripto_tab = cripto_tab.withColumn('precio_btc_corta', criptos_prices.prom_btc_corta.cast('float'))
cripto_tab = cripto_tab.withColumn('precio_usd_corta', criptos_prices.prom_usd_corta.cast('float'))
cripto_tab = cripto_tab.withColumn('precio_eur_corta', criptos_prices.prom_eur_corta.cast('float'))


print(cripto_tab)

#Inicializamos una query 
writeBitcoin = cripto_tab.writeStream.format("memory"). \
    queryName("bitcoinquery"). \
    trigger(processingTime='2 seconds'). \
    start()

DataFrame[prom_btc: string, prom_usd: string, prom_eur: string, btc: string, usd: string, eur: string, prom_btc_corta: string, prom_usd_corta: string, prom_eur_corta: string, timestamp: timestamp, precio_btc: float, precio_usd: float, precio_eur: float, btc_float: float, usd_float: float, eur_float: float, precio_btc_corta: float, precio_usd_corta: float, precio_eur_corta: float]


## Muestra de los datos para el valor del Ethereum en USD
### Valor Real, Señal Larga, Señal Corta

In [0]:
%sql
select usd_float, precio_usd, precio_usd_corta  from bitcoinquery

usd_float,precio_usd,precio_usd_corta
2027.67,2028.0753,2028.219
2026.84,2028.08,2028.154
2028.21,2028.0537,2028.017
2028.09,2028.0734,2028.003
2028.16,2028.085,2027.977
2026.56,2028.106,2027.975
2026.98,2028.0724,2027.8
2028.09,2028.0507,2027.667
2026.84,2028.068,2027.708
2026.9,2028.038,2027.571


Mostramos los datos para los euros en el mismo formato y significado que la celda anterior;

In [0]:
%sql
select eur_float, precio_eur, precio_eur_corta  from bitcoinquery

eur_float,precio_eur,precio_eur_corta
1913.81,1914.213,1914.168
1913.19,1914.1913,1914.092
1913.7,1914.149,1913.973
1913.19,1914.134,1913.936
1913.75,1914.102,1913.869
1913.19,1914.0886,1913.806
1913.75,1914.0697,1913.739
1913.7,1914.068,1913.708
1913.19,1914.053,1913.64
1913.06,1914.007,1913.521


Y finalmente mostramos también los valores del BTC

In [0]:
%sql
select btc_float, precio_btc, precio_btc_corta  from bitcoinquery

btc_float,precio_btc,precio_btc_corta
0.06659,0.06657533,0.066582
0.06658,0.06657633,0.066583
0.06659,0.066577,0.066582
0.06659,0.066577666,0.066584
0.06659,0.066578,0.066586
0.06658,0.066579,0.066586
0.06659,0.06657967,0.066586
0.06659,0.06658,0.066587
0.06658,0.066581,0.066587
0.06658,0.066581,0.066586


## Análisis de Resultados

El algoritmo de ventanas deslizantes muestra la potencia de los algoritmos sencillos, pues aún cuando se basa únicamente en la media aritmética, esta idea sigue estando presente al momento de hacer *trading* sobre crypto monedas, stocks de mercado, NFTs, etc. 

Cuando se realiza el análisis técnico sobre un stock, uno de los primeros indicadores a evaluar son las medias móviles con el fin de intentar *seguir la tendencia* para poder concretar acciones de venta o compra sobre el volúmen de stock dentro de nuestro portafolio. 

Más aún, las medias móviles persisten no sólo en indicadores de acciones, sino en estrategias de trading como el cruce dorado o el cruce de la muerte. 

Considerando que a través de Apache, PySpark y el API de algún exchange se tiene la facilidad de obtener y analizar datos en tiempo real, el proyecto puede evolucionar en;

- Una implementación más robusta de Medias Móviles para el análisis técnico de precios de acciones y otros indicadores. 

- Una gran fuente de conjuntos de datos para el desarrollo de modelos de Machine Learning, tales como LSTM o Transformers sobre Series de Tiempo que sean capaces de predecir con relativamente buena precisión los costos del mercado y ejecutar acciones de compra-venta. 

- Implementación de modelos de detección de opportunidades de Arbitrage, (bajo el algoritmo de Bellman-Ford para la detección de ciclos negativos).

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e588c84d-c012-4c07-8593-b08d1bf1b087' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>